In [ ]:
import pandas as pd
import os
import tempfile
from typing import Dict, Texte
import numpy as np
import tensorflow as tf
from collections import defaultdict

In [ ]:
!pip install tensorflow_recommenders

In [ ]:
import tensorflow_recommenders as tfrs

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# trdata = pd.read_csv('/content/drive/MyDrive/Assignment 2/train_0.txt', sep="\t", header=None)
# trdata = trdata.rename(columns={0:'user_id', 1:'song_id', 2:'rating'})
tdata = pd.read_csv('/content/drive/MyDrive/Assignment 2/test_0.txt', sep="\t", header=None)
tdata = tdata.rename(columns={0:'user_id', 1:'song_id', 2:'rating'})

In [ ]:
tdata.head(10)

,user_id,song_id,rating
0,0,7171,5
1,0,8637,4
2,0,21966,4
3,0,35821,5
4,0,82446,5
5,0,90409,5
6,0,107410,5
7,0,131919,5
8,0,132685,3
9,0,136507,3


In [ ]:
tr_df = trdata
t_df = tdata

In [ ]:
songs = pd.read_csv('/content/drive/MyDrive/Assignment 2/song-attributes.txt', sep="\t", header=None)
songs = songs.rename(columns={0:'song_id', 1:'album_id', 2:'artist_id', 3:'genre_id'})

In [ ]:
genres = pd.read_csv('/content/drive/MyDrive/Assignment 2/genre-hierarchy.txt', sep="\t", header=None)
genres = genres.rename(columns={0:'genre_id', 1:'parent_genre', 2:'level', 3:'genre_name'})

In [ ]:
df = tr_df.merge(songs, on='song_id').merge(genres, on='genre_id')

In [ ]:
df

,user_id,song_id,rating,album_id,artist_id,genre_id,parent_genre,level,genre_name
0,0,7171,5,19948,2889,0,0,1,Unknown
1,4379,7171,2,19948,2889,0,0,1,Unknown
2,13442,7171,5,19948,2889,0,0,1,Unknown
3,13946,7171,5,19948,2889,0,0,1,Unknown
4,17215,7171,5,19948,2889,0,0,1,Unknown
...,...,...,...,...,...,...,...,...,...
1999995,121551,49230,1,13966,6084,213,211,2,Lounge
1999996,160801,49230,3,13966,6084,213,211,2,Lounge
1999997,161666,49230,5,13966,6084,213,211,2,Lounge
1999998,107805,122805,1,695,75,113,106,2,New Wave


In [ ]:
genres.head(20)

,genre_id,parent_genre,level,genre_name
0,0,0,1,Unknown
1,1,1,1,Electronic/Dance
2,2,1,2,Ambient
3,3,2,3,Ambient Dub
4,4,2,3,Ambient Tech
5,5,1,2,Breaks
6,6,1,2,Jungle / Drum &amp; Bass
7,7,1,2,House
8,8,7,3,Deep House
9,9,7,3,Tech House


## Model 1

In [ ]:
t_df = t_df.sample(frac=1)
ratingsTrain = t_df.values[:1_800_000]
ratingsTest = t_df.values[1_800_000:]

In [ ]:

ratingsPerUser = defaultdict(list)
ratingsPerSong = defaultdict(list)
for i in ratingsTrain:
  u = i[0]
  s = i[1]
  r = i[2]
  ratingsPerUser[u].append((s, r))
  ratingsPerSong[s].append((u, r))


In [ ]:
trainRatings = [r[2] for r in ratingsTrain]
globalAverage = sum(trainRatings) * 1.0 / len(trainRatings)
globalAverage

3.459966666666667

In [ ]:
validMSE = 0
for i in ratingsTrain:
  u = i[0]
  s = i[1]
  r = i[2]
  se = (r - globalAverage)**2
  validMSE += se

validMSE /= len(ratingsTrain)

print("Validation MSE (average only) = " + str(validMSE))

Validation MSE (average only) = 2.4136917766708796


In [ ]:
betaU = {}
betaI = {}
for u in ratingsPerUser:
    betaU[u] = 0

for s in ratingsPerSong:
    betaI[s] = 0
alpha = globalAverage

In [ ]:
def iterate(lamb):
    newAlpha = 0
    for u, s, r in ratingsTrain:
        newAlpha += r - (betaU[u] + betaI[s])
    alpha = newAlpha / len(ratingsTrain)
    for u in ratingsPerUser:
        newBetaU = 0
        for s,r in ratingsPerUser[u]:
            newBetaU += r - (alpha + betaI[s])
        betaU[u] = newBetaU / (lamb + len(ratingsPerUser[u]))
    for s in ratingsPerSong:
        newBetaI = 0
        for u,r in ratingsPerSong[s]:
            newBetaI += r - (alpha + betaU[u])
        betaI[s] = newBetaI / (lamb + len(ratingsPerSong[s]))
    mse = 0
    for u,s,r in ratingsTrain:
        prediction = alpha + betaU[u] + betaI[s]
        mse += (r - prediction)**2
    regularizer = 0
    for u in betaU:
        regularizer += betaU[u]**2
    for s in betaI:
        regularizer += betaI[s]**2
    mse /= len(ratingsTrain)
    return mse, mse + lamb*regularizer

In [ ]:
mse,objective = iterate(1)
newMSE,newObjective = iterate(1)
iterations = 2

In [ ]:

iterations = 1
while iterations < 10 or objective - newObjective > 0.0001:
    mse, objective = newMSE, newObjective
    newMSE, newObjective = iterate(5)
    iterations += 1
    print("Objective after " + str(iterations) + " iterations = " + str(newObjective))
    print("MSE after " + str(iterations) + " iterations = " + str(newMSE))

Objective after 2 iterations = 438789.32492574747
MSE after 2 iterations = 1.3954868824135989
Objective after 3 iterations = 431085.61936801457
MSE after 3 iterations = 1.3973506715919737
Objective after 4 iterations = 430028.5992980959
MSE after 4 iterations = 1.397328561508256
Objective after 5 iterations = 429671.9788968218
MSE after 5 iterations = 1.397107187720516
Objective after 6 iterations = 429499.5641593056
MSE after 6 iterations = 1.3969073178571605
Objective after 7 iterations = 429411.5966305584
MSE after 7 iterations = 1.396747490673531
Objective after 8 iterations = 429371.8360161693
MSE after 8 iterations = 1.3966222362667693
Objective after 9 iterations = 429361.12262195
MSE after 9 iterations = 1.3965241611915864
Objective after 10 iterations = 429367.74323210464
MSE after 10 iterations = 1.396447040022957


In [ ]:
validMSE = 0
predictions = []
for u,b,r in ratingsTest:
    bu = 0
    bi = 0
    if u in betaU:
        bu = betaU[u]
    if b in betaI:
        bi = betaI[b]
    prediction = alpha + bu + bi
    predictions.append(prediction)
    validMSE += (r - prediction)**2

validMSE /= len(ratingsTest)
print("Validation MSE = " + str(validMSE))


Validation MSE = 1.7592670307297835


In [ ]:
m1pred = predictions

##Model 2

In [ ]:
#Randomly shuffle train data and convert values to str
tr_df = tr_df.sample(frac=1)
tr_df['user_id'] = tr_df['user_id'].astype(str)
tr_df['song_id'] = tr_df['song_id'].astype(str)
tr_df['rating'] = tr_df['rating'].astype(str)
#Randomly shuffle test data and convert values to str
t_df = t_df.sample(frac=1)
t_df['user_id'] = t_df['user_id'].astype(str)
t_df['song_id'] = t_df['song_id'].astype(str)
t_df['rating'] = t_df['rating'].astype(str)

In [ ]:
#Convert train and test data to tensors
tr_data = tf.convert_to_tensor(np.array(tr_df[['user_id', 'song_id', 'rating']]))
t_data = tf.convert_to_tensor(np.array(t_df[['user_id', 'song_id', 'rating']]))

In [ ]:
#convert_to_tensor only takes values with one dtype, re-convert ratings values to ints and add all values as dict entries
tr_dataset = tf.data.Dataset.from_tensor_slices(tr_data)
tr_dataset = tr_dataset.map(lambda x: {
    "song_id": x[1],
    "user_id": x[0],
    "user_rating": int(x[2])
})
t_dataset = tf.data.Dataset.from_tensor_slices(t_data)
t_dataset = t_dataset.map(lambda x: {
    "song_id": x[1],
    "user_id": x[0],
    "user_rating": int(x[2])
})

In [ ]:
#seperate user_ids and song_ids for vocabulary generation
user_ids = tr_dataset.batch(1_000_000).map(lambda x: x['user_id'])
song_ids = tr_dataset.batch(1_000_000).map(lambda x: x['song_id'])
#find unique song and user ids to create vocabulary 
unique_song_ids = np.unique(np.concatenate(list(song_ids)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

In [ ]:
len(unique_user_ids)

200000

In [ ]:
class RankingModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        embedding_dimension = 32

        self.user_embeddings = tf.keras.Sequential([
            tf.keras.layers.experimental.preprocessing.StringLookup(
            vocabulary=unique_user_ids, mask_token=None),
            tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
        ])
        
        self.song_embeddings = tf.keras.Sequential([
            tf.keras.layers.experimental.preprocessing.StringLookup(
            vocabulary=unique_song_ids, mask_token=None),
            tf.keras.layers.Embedding(len(unique_song_ids) + 1, embedding_dimension)
        ])
        self.ratings = tf.keras.Sequential([
            tf.keras.layers.Dense(256, activation='relu'),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(1)
        ])
        
    def call(self, inputs):

        user_id, song_id = inputs

        user_embedding = self.user_embeddings(user_id)
        song_embedding = self.song_embeddings(song_id)

        return self.ratings(tf.concat([user_embedding, song_embedding], axis=1))

In [ ]:
class SongModel(tfrs.models.Model):
    def __init__(self):
        super().__init__()
        self.ranking_model: tf.keras.Model = RankingModel()
        self.task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
            loss = tf.keras.losses.MeanSquaredError(),
            metrics=[tf.keras.metrics.RootMeanSquaredError()]
        )
    def call(self, features: Dict[str, tf.Tensor]) -> tf.Tensor:
        return self.ranking_model(
            (features["user_id"], features["song_id"]))
    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
        rating_predictions = self.ranking_model((features['user_id'], features['song_id']))
        return self.task(labels=features['user_rating'], predictions=rating_predictions)
    

In [ ]:
model = SongModel()
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.3))
cached_train = tr_dataset.shuffle(100_000).batch(8192).cache()
cached_test = t_dataset.batch(4096).cache()
model.fit(cached_train, epochs = 6)

Epoch 1/6
9320/9320 [==============================] - 1106s 118ms/step - root_mean_squared_error: 1.2701 - loss: 1.6131 - regularization_loss: 0.0000e+00 - total_loss: 1.6131
Epoch 2/6
9320/9320 [==============================] - 42s 4ms/step - root_mean_squared_error: 1.2180 - loss: 1.4836 - regularization_loss: 0.0000e+00 - total_loss: 1.4836
Epoch 3/6
9320/9320 [==============================] - 42s 4ms/step - root_mean_squared_error: 1.1975 - loss: 1.4339 - regularization_loss: 0.0000e+00 - total_loss: 1.4339
Epoch 4/6
9320/9320 [==============================] - 42s 4ms/step - root_mean_squared_error: 1.1845 - loss: 1.4030 - regularization_loss: 0.0000e+00 - total_loss: 1.4030
Epoch 5/6
9320/9320 [==============================] - 42s 4ms/step - root_mean_squared_error: 1.1703 - loss: 1.3696 - regularization_loss: 0.0000e+00 - total_loss: 1.3696
Epoch 6/6
9320/9320 [==============================] - 42s 4ms/step - root_mean_squared_error: 1.1542 - loss: 1.3322 - regularization_lo

In [ ]:
model.evaluate(cached_test, return_dict=True)

489/489 [==============================] - 22s 44ms/step - root_mean_squared_error: 1.1815 - loss: 1.3955 - regularization_loss: 0.0000e+00 - total_loss: 1.3955


{'root_mean_squared_error': 1.1815451383590698,
 'loss': 1.226922869682312,
 'regularization_loss': 0,
 'total_loss': 1.226922869682312}

In [ ]:
tf.saved_model.save(model, "export")

In [ ]:
!zip -r /content/file.zip "/content/export"

  adding: content/export/ (stored 0%)
  adding: content/export/assets/ (stored 0%)
  adding: content/export/saved_model.pb (deflated 76%)
  adding: content/export/variables/ (stored 0%)
  adding: content/export/variables/variables.index (deflated 62%)
  adding: content/export/variables/variables.data-00000-of-00001 (deflated 17%)


In [ ]:
test_ratings = {}
test_song_ids = ['82446']
for song_id in test_song_ids:
    test_ratings[song_id] = model({
        'user_id':np.array(['0']),
        'song_id':np.array([song_id])
    })
print("Ratings:")
for title, score in sorted(test_ratings.items(), key=lambda x: x[1], reverse=True):
  print(f"{title}: {score}")

Ratings:
82446: [[4.270583]]


In [ ]:
scores = {}
for i in t_df['song_id'].unique():
  scores[i] = np.array(model({
      'user_id':np.array(['18385']),
      'song_id':np.array([i])
  }))[0][0]

In [ ]:
sscores = sorted(scores, key = scores.get, reverse=True)
sorted_dict = {}
for i in sscores:
  sorted_dict[i] = scores[i]


In [ ]:
relgenres = []
for i in sscores[:1000]:
  relgenres.append(df[df['song_id'] == int(i)].genre_name.iloc[0])

In [ ]:
pd.Series([i for i in relgenres if i != 'Unknown']).value_counts()

Rock                 122
Speed Metal           20
Folk                  13
Rap                   13
Pop                    9
Modern Rock            8
Reggae                 4
Country                1
Movie Soundtracks      1
Classic R&B            1
Adult Alternative      1
R&B                    1
dtype: int64

In [ ]:
u = df[df['user_id'] == 18385]
u[u['genre_name'] != "Unknown"]

,user_id,song_id,rating,album_id,artist_id,genre_id,parent_genre,level,genre_name
66487605,18385,21255,1,5373,2402,134,134,1,Rock
66613324,18385,79249,1,10014,6679,134,134,1,Rock
67260983,18385,60721,1,9521,4149,134,134,1,Rock
67531751,18385,6739,5,15468,6916,134,134,1,Rock
67655747,18385,49103,1,3947,5888,134,134,1,Rock
68128627,18385,91576,5,14794,7595,134,134,1,Rock
68551225,18385,55598,4,15188,2022,134,134,1,Rock
68633848,18385,44748,1,10121,2183,134,134,1,Rock
68693834,18385,85299,1,6969,7648,134,134,1,Rock
69683656,18385,75660,1,1553,4058,134,134,1,Rock


In [ ]:
u = t_df[t_df['user_id'] == '18385']
u

,user_id,song_id,rating
183858,18385,87644,3
183856,18385,80344,3
183852,18385,36657,1
183857,18385,82358,5
183850,18385,22342,3
183853,18385,51271,1
183855,18385,60465,1
183851,18385,30704,4
183854,18385,60042,1
183859,18385,94762,3


In [ ]:
import zipfile
with zipfile.ZipFile("file.zip","r") as zip_ref:
  zip_ref.extractall("export")

In [ ]:
loaded = tf.saved_model.load("/content/export/content/export")

In [ ]:
loaded

<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject at 0x7fe3a59d8f50>

In [ ]:
mod1preds = []
for i in ratingsTest:
    song_id = str(i[1])
    user_id = str(i[0])
    mod1preds.append(np.array(loaded({
        'user_id':np.array([user_id]),
        'song_id':np.array([song_id])
    }))[0][0])

In [ ]:
cdf = pd.DataFrame()
cdf['song/user'] = [(x[1], x[0]) for x in ratingsTest]
cdf = cdf.set_index('song/user')
cdf['model_1_predictions'] = predictions
cdf['model_2_predictions'] = mod2preds
cdf['actual'] = [x[2] for x in ratingsTest]

In [ ]:
cdf.sample(frac=1).head(20)

,model_1_predictions,model_2_predictions,actual
song/user,,,
"(55815, 144793)",4.505848,4.490824,5
"(41036, 545)",2.432631,2.723452,3
"(94982, 140481)",4.311240,1.896512,1
"(71055, 132736)",3.484031,3.567280,5
"(74890, 113275)",3.253808,3.300029,2
"(29753, 186323)",4.543764,4.029703,4
"(31496, 194251)",3.110066,2.330466,2
"(92981, 40737)",3.743877,2.796898,3
"(112082, 166888)",2.557342,3.645494,5
